In [81]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import codecs
import time
import re
import os
from IPython.display import display, HTML
from difflib import SequenceMatcher
import sys
sys.path 

['',
 'C:\\anaconda\\python36.zip',
 'C:\\anaconda\\DLLs',
 'C:\\anaconda\\lib',
 'C:\\anaconda',
 'C:\\anaconda\\lib\\site-packages',
 'C:\\anaconda\\lib\\site-packages\\Babel-2.5.0-py3.6.egg',
 'C:\\anaconda\\lib\\site-packages\\win32',
 'C:\\anaconda\\lib\\site-packages\\win32\\lib',
 'C:\\anaconda\\lib\\site-packages\\Pythonwin',
 'C:\\anaconda\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\k.l.s.koutilya varma\\.ipython']

In [82]:
# sys.path.append('/sanskrit_transcoder')
# import transcoder


In [83]:

import importlib.util
spec = importlib.util.spec_from_file_location("module.name","sanskrit_transcoder/transcoder.py")
transcoder = importlib.util.module_from_spec(spec)
spec.loader.exec_module(transcoder)
transcoder.transcoder_set_dir('sanskrit_transcoder/data/transcoder');

In [183]:
names = ["workdata/lines_submit/correct_lines/sorteddata/","workdata/lines_submit/wrong_lines/sorteddata/"]
fname1 = "workdata/lines_submit/correct_lines/sorteddata/1/dataframe"
fname2 = "workdata/lines_submit/correct_lines/sorteddata/1/dataframe_slp1"
fnames = os.listdir("workdata/lines_submit/correct_lines/sorteddata/")
for name in names :
    fnames = os.listdir(name)
    for fname in fnames:
        if fname == ".ipynb_checkpoints":
            continue
#         print(fname+"/dataframe.txt")
        df = pd.read_csv(name+fname+"/dataframe.txt",encoding = 'utf-8' ,sep = ',')
        df['word_slp1'] = df['word']
        for i in df.index:
            df.loc[i,'word_slp1']   =  transcoder.transcoder_processString(df.loc[i,'word'],'roman','slp1')
        df['sandhi_indexs'] = ""
        df['sandhi_words'] = ""
        for i in df.index : 
            word1 = df.loc[i,'word_slp1']
            chunkno = df.loc[i,'chunk_no']
            position1 =  df.loc[i,'position']
            level = df.loc[i,'level']
            clspan1 = df.loc[i,'colspan']
            df1 = df.loc[(df['chunk_no'] == chunkno) & (df['level']!=level)]
            for j in df1.index :
                position2 = df1.loc[j,'position']
                word2 = df1.loc[j,'word_slp1']
                word3 = df1.loc[j,'word']
                if(position2 == position1 + clspan1-1):
                    if(len(s.loc[(s['c1']==word1[-1:] )& (s['c2']==word2[:1])])):
                        
                        df.loc[i,'sandhi_indexs'] += str(j)+", "
                        df.loc[i,'sandhi_words'] += word3+"("+word2+"), "
                    elif(word1[-1:]=='H'):
                        if(len(s.loc[(s['c1']==word1[-2:] )& (s['c2']==word2[:1])])):
                        
                            df.loc[i,'sandhi_indexs'] += str(j)+", "
                            df.loc[i,'sandhi_words'] += word3+"("+word2+"), "
        
        df.to_csv(name+fname+"/dataframe_withsandhi.txt" ,encoding = 'utf-8' , sep=',',index = False,mode = 'w' )
                        
                        
#         break
#     break
# print(name+fname+"/dataframe_withsandhi.txt")

In [187]:
for name in names :
    fnames = os.listdir(name)
    for fname in fnames:
        if fname == ".ipynb_checkpoints":
            continue
        df1 = pd.read_csv(name+fname+"/dataframe.txt",encoding = 'utf-8' ,sep = ',')
        df2 = df1 = pd.read_csv(name+fname+"/dataframe_withsandhi.txt",encoding = 'utf-8' ,sep = ',')
        df1.to_csv(name+fname+"/dataframe.csv" ,encoding = 'utf-8' , sep=',',index = False,mode = 'w' )
        df2.to_csv(name+fname+"/dataframe_withsandhi.csv" ,encoding = 'utf-8' , sep=',',index = False,mode = 'w' )

In [121]:
    # df['word_slp1'] = df['word']
    # for i in df.index:
    #     df.loc[i,'word_slp1']   =  transcoder.transcoder_processString(df.loc[i,'word'],'roman','slp1')


,id,level,color_class,position,chunk_no,word,lemma,pre_verb,morph,colspan,wordlenth,aux_inf,word_slp1
0,0,1,mauve_back,0,1,ekadā,ekadā,NaN,adv.,5,5,NaN,ekadA
1,1,1,mauve_back,6,1,yadā,yadā,NaN,adv.,4,4,NaN,yadA
2,2,1,light_blue_back,11,2,saḥ,tad,NaN,nom. sg. m.,3,3,NaN,saH
3,3,1,deep_sky_back,15,3,sabhāyām,sabhā,NaN,loc. sg. f.,7,8,NaN,saBAyAm
4,4,1,carmin_back,23,4,upāviśat,viś,upa,impft. [6] ac. sg. 3,8,8,sence of lemma = 1,upAviSat
5,5,1,grey_back,32,5,tadā_ekaḥ,tadā,NaN,?,9,9,sence of lemma = ekaḥ,tadA_ekaH
6,6,1,yellow_back,42,6,rāja,rājan,NaN,iic.,4,4,NaN,rAja
7,7,1,deep_sky_back,46,6,puruṣaḥ,puruṣa,NaN,nom. sg. m.,7,7,NaN,puruzaH
8,8,1,light_blue_back,54,7,ekam,eka,NaN,acc. sg. m.,4,4,NaN,ekam
9,9,1,light_blue_back,54,7,ekam,eka,NaN,acc. sg. n.,4,4,NaN,ekam


In [182]:
# s = pd.read_csv("all_sandhi.txt",encoding = 'utf-8' ,sep = ',')
# s.loc[(s['c1']=='A' )& (s['c2']=='A')]
# display(s.loc(s['c1']=='H'))

In [148]:
# for i in df.index : 
#     word1 = df.loc[i,'word_slp1']
#     chunkno = df.loc[i,'chunk_no']
#     position1 =  df.loc[i,'position']
#     level = df.loc[i,'level']
#     clspan1 = df.loc[i,'colspan']
#     df1 = df.loc[(df['chunk_no'] == chunkno) & (df['level']!=level)]
#     for j in df1.index :
#         position2 = df1.loc[j,'position']
#         word2 = df1.loc[j,'word_slp1']
#         if(position2 == position1 + clspan1-1):
#             if(len(s.loc[(s['c1']==word1[-1:] )& (s['c2']==word2[:1])])):
#                 print(str(j)+"--->"+word1+"--"+word2)
#                 display(s.loc[(s['c1']==word1[-1:] )& (s['c2']==word2[:1])])

52--->saBA--ayAm


,id,c1,c2,c3
48,48,A,a,A


53--->saBA--AyAm


,id,c1,c2,c3
49,49,A,A,A


54--->saBA--ayAm


...

,id,c1,c2,c3
48,48,A,a,A


53--->BA--AyAm


,id,c1,c2,c3
49,49,A,A,A


54--->BA--ayAm


,id,c1,c2,c3
48,48,A,a,A


In [129]:
#  df1 = df.loc[(df['chunk_no']==1) & (df['level']!=1)]
# y = df.loc[(df['chunk_no']==1) & (df['level']!=1)].index
# for i in y:
#     display(df1.loc[i])

id                      19
level                    2
color_class    yellow_back
position                 0
chunk_no                 1
word                   eka
lemma                  eka
pre_verb               NaN
morph                 iic.
colspan                  3
wordlenth                3
aux_inf                NaN
word_slp1              eka
Name: 19, dtype: object

id                      20
level                    2
color_class      cyan_back
position                 3
chunk_no                 1
word                   dāḥ
lemma                   da
pre_verb               NaN
morph          nom. pl. m.
colspan                  3
wordlenth                3
aux_inf                NaN
word_slp1              dAH
Name: 20, dtype: object

id                      21
level                    2
color_class      cyan_back
position                 3
chunk_no                 1
word                   dāḥ
lemma                   da
pre_verb               NaN
morph          acc. pl. f.
colspan                  3
wordlenth                3
aux_inf                NaN
word_slp1              dAH
Name: 21, dtype: object

id                      22
level                    2
color_class      cyan_back
position                 3
chunk_no                 1
word                   dāḥ
lemma                   da
pre_verb               NaN
morph          nom. pl. f.
colspan                  3
wordlenth                3
aux_inf                NaN
word_slp1              dAH
Name: 22, dtype: object

id                      35
level                    3
color_class      cyan_back
position                 3
chunk_no                 1
word                    dā
lemma                   da
pre_verb               NaN
morph          nom. sg. f.
colspan                  2
wordlenth                2
aux_inf                NaN
word_slp1               dA
Name: 35, dtype: object

In [171]:
# df['e'] = np.nan
# df.loc[0,'e'] +=str(1)
# df.loc[0,'e'] +=1
# df

TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')